# Semana 1 – Fundamentos RAG con Dataset de Recetas

**Objetivo:**  
Practicar la generación de embeddings y la búsqueda semántica de información externa.

**Dataset:**  
- Recetas en español (title, ingredients, steps, url, uuid)  
- Fuente: Hugging Face  
- Tamaño: ~1300 recetas

**Qué se hará en este notebook:**  
1. Leer y explorar el dataset con Pandas.  
2. Preprocesar los textos para generar embeddings.  
3. Generar embeddings usando Sentence-Transformers.  
4. Indexar los embeddings en FAISS para búsqueda semántica.  
5. Hacer consultas de prueba y recuperar las recetas más relevantes.

**Notas:**
*   pandas → leer CSV y manipular dataframes.
*   numpy → operaciones vectoriales.
*   sentence-transformers → generar embeddings.
*   faiss-cpu → búsqueda vectorial eficiente.

In [ ]:
!pip install pandas numpy sentence-transformers faiss-cpu

In [4]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
# file path in the colab's root directory
csv_path = '/recipes_dataset.csv'

In [17]:
# check the content
df = pd.read_csv(csv_path)
df.head(3)

,title,url,ingredients,steps,uuid
0,Arepas de Queso,https://www.mycolombianrecipes.com/es/arepas-d...,1 taza de harina de arepa blanca o amarilla\r\...,"Combine la harina de maÃ­z, agua caliente, el ...",86af61e4-e16a-11ed-9591-a96d6180cd25
1,Sudado de Pollo,https://www.mycolombianrecipes.com/es/sudado-d...,8 muslos de pollo sin la piel\r\n1 cucharada d...,"En una olla grande, caliente el aceite vegetal...",86af61e5-e16a-11ed-abef-a96d6180cd25
2,Sancocho TrifÃ¡sico,https://www.mycolombianrecipes.com/es/sancocho...,1 taza de cebolla picada\r\n1 pimientÃ³n rojo ...,"Coloque la cebolla, el pimientÃ³n, el ajo y el...",86af61e6-e16a-11ed-bcf5-a96d6180cd25


## 1. Preprocesamiento de texto

In [ ]:
# create a unique column with title + ingredients + steps. This helps the embedding to geet all the relevant information.
# it is important to mention that there are many ways to do this because it depends on the case and this is the simplest way

df["text"] = df["title"] + ". Ingredientes: " + df["ingredients"] + ". Pasos: " + df["steps"]

In [19]:
#check an example
df[["title", "text"]].head(1)

,title,text
0,Arepas de Queso,Arepas de Queso. Ingredientes: 1 taza de harin...


## 2. Generación de embeddings
* Como resultado cada fila del dataset tendrá un embedding en **vector numérico**.
* Estos embeddings son los que se indexan para realizar la búsqueda semántica.

Link del modelo utilizado: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [20]:
# load the model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [21]:
# generate embeddings
embeddings = model.encode(df["text"].tolist(), show_profress_bar=True)

In [22]:
# show embeddings size
print(f"Embedding size: {embeddings.shape}")

Embedding size: (99, 384)


#### Explicación de 'shape' (99, 384)
* 99 → cantidad de registros procesados (el df con 99 recetas).
* 384 → dimensión de cada embedding, o sea el tamaño del vector numérico que representa cada receta.

##### Qué significa la dimensión (384)

- Cada embedding es un vector en un espacio de 384 dimensiones.
- Cada número dentro del vector representa una “característica” del texto según lo aprendió el modelo all-MiniLM-L6-v2.
- Estos 384 números juntos permiten medir similitud semántica entre textos usando distancias (coseno, Euclidiana, etc.).

## 3. Indexación con FAISS
Con esta indexacion se pueden hacer consultas usando búsqueda por similaridad; puede ser euclidiana o por coseno.

In [ ]:
# embedding dimension
dim = embeddings.shape[1] #384

# create index
index = faiss.IndexFlatL2(dim) # L2 = euclidian distance (menor distancia = más cercano / más parecido semánticamente)

In [29]:
print(type(np.array(embeddings, dtype="float32")))

<class 'numpy.ndarray'>


In [30]:
# add vectors to index
index.add(np.array(embeddings, dtype="float32"))

In [32]:
print(f"Vectors quantity: {index.ntotal}") #99

Vectors quantity: 99


## 4. Test queries
Dataset externo → embeddings → FAISS → recuperación semántica. (sin LLM)

In [105]:
query = "bandeja paisa"

# encode query
query_vec = model.encode([query])

In [106]:
# SEARCH only 5 -> k = 5
# distancias entre el vector de la query y los vectores más cercanos.
# índices de los registros más cercanos en el DataFrame.

distances, records_index = index.search(np.array(query_vec, dtype="float32"), k=5)

In [107]:
print(distances)

[[1.0703467 1.3371392 1.3414571 1.3536253 1.3541741]]


In [108]:
print(records_index)

[[56  4 11 15 71]]


In [109]:
records_index[0] # array([21, 20, 39])

array([56,  4, 11, 15, 71])

In [112]:
for idx in records_index[0]:
  print(df.iloc[idx]["title"])
  print(df.iloc[idx]["ingredients"])
  print(df.iloc[idx]["steps"])
  print("\n")

Bandeja Paisa
Frijoles AntioqueÃ±os
Arroz Blanco
Carne en polvo
Chicharrones 
Chorizos Cocidos
Huevos Fritos
Tajadas de PlÃ¡tano
Aguacate
Hogao
Preparar los frijoles, hogao y carne en polvo el dÃ­a danterior y guardar en la nevera.
Cuando este listo para servir la bandeja paisa, caliente los frijoles ,hogao, la carne en polvo y prepare los chicharrones.
Cocine el arroz blanco y los plÃ¡tanos. Â FreÃ­r los huevos y chorizos.


Sopa de Lentejas Colombiana
1 cucharadita de aceite vegetal
1 taza de chorizo en rodajas
Â½ taza de cebolla picada
1 diente de ajo picado
Â½ taza de cebolla larga picada
Â½ taza de tomates picados
5 tazas de agua
1 taza de lentejas
Â½ cucharadita de sal
Â½ cucharadita de pimienta
Â½ taza de zanahorias ralladas
Â½ taza de papas cortadas en trozos
Â½ cucharadita de comino en polvo
Cocinar el chorizo en el aceite en una olla grande a fuego medio, revolviendo de vez en cuando, unos 5 minutos. Con una cuchara ranurada, transfiera el chorizo a un plato cubierto con pape

### Example with Cosine

In [92]:
# normalizar embeddings para que el producto sea equivalente a cosine similarity
embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [ ]:
dim = embeddings.shape[1] # 384

# create index
index_cos = faiss.IndexFlatIP(dim) # cosine (más cercano a 1 = más parecido)
index_cos.add(np.array(embeddings_norm, dtype="float32"))

In [94]:
# Normalizar la query
query_vec_norm = query_vec / np.linalg.norm(query_vec, axis=1, keepdims=True)

In [95]:
# buscar las 5
similarities, indices = index_cos.search(np.array(query_vec_norm, dtype="float32"), k=5)

In [96]:
similarities

array([[0.47808105, 0.45292142, 0.4106421 , 0.40561402, 0.4047767 ]],
      dtype=float32)

In [97]:
indices

array([[21, 92, 64, 63, 42]])

In [98]:
for indice in indices[0]:
  print(df.iloc[indice])

title                                    Sopa de Arroz con Pollo
url            https://www.mycolombianrecipes.com/es/sopa-de-...
ingredients    8 muslos de pollo o 4 pechugas\r\n7 tazas de a...
steps          En una olla grande colocar todos los ingredien...
uuid                        86af71cd-e16a-11ed-b708-a96d6180cd25
text           Sopa de Arroz con Pollo. Ingredientes: 8 muslo...
Name: 21, dtype: object
title                             Sopa de Pollo y PlÃ¡tano Verde
url            https://www.mycolombianrecipes.com/es/sopa-de-...
ingredients    2 pechugas de pollo con hueso y sin piel\r\n10...
steps          Coloque el pollo en una olla grande con el com...
uuid                        86af7214-e16a-11ed-b863-a96d6180cd25
text           Sopa de Pollo y PlÃ¡tano Verde. Ingredientes: ...
Name: 92, dtype: object
title                                      Sopa de Pollo y Pasta
url            https://www.mycolombianrecipes.com/es/sopa-de-...
ingredients    6 muslos de pollo con hueso